In [ ]:
import torch
from torch.utils.data import DataLoader
from torch import nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score, matthews_corrcoef, recall_score, precision_score
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.metrics import roc_curve
import torch.nn.functional as F
import xgboost as xgb
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [ ]:
# get roc_auc, metrics_sn, metrics_sp, metrics_ACC, metrics_F1, metrics_MCC
def calculate_multiclass_metrics(true_labels, predicted_labels, predicted_probabilities, num_classes):
    accuracy = accuracy_score(true_labels, predicted_labels)
    mcc = matthews_corrcoef(true_labels, predicted_labels)
    
    sensitivity_per_class = []
    specificity_per_class = []
    auc_per_class = []
    f1_per_class = []

    for i in range(num_classes):
        true_binary = (np.array(true_labels) == i).astype(int)
        pred_binary = (np.array(predicted_labels) == i).astype(int)

        cm = confusion_matrix(true_binary, pred_binary, labels=[0, 1])
        tn, fp, fn, tp = cm.ravel()

        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        sensitivity_per_class.append(sensitivity)
        specificity_per_class.append(specificity)

        
        auc = roc_auc_score(true_binary, predicted_probabilities[:, i]) if len(np.unique(true_binary)) > 1 else 0
        auc_per_class.append(auc)

        f1 = f1_score(true_binary, pred_binary) if len(np.unique(true_binary)) > 1 else 0
        f1_per_class.append(f1)

    avg_sensitivity = np.mean(sensitivity_per_class)
    avg_specificity = np.mean(specificity_per_class)
    avg_auc = np.mean(auc_per_class) if auc_per_class else 0
    avg_f1 = np.mean(f1_per_class)

    print(f"Average AUC: {avg_auc:.4f}")
    print(f"Average Sensitivity: {avg_sensitivity:.4f}")
    print(f"Average Specificity: {avg_specificity:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Average F1-score: {avg_f1:.4f}")
    print(f"Matthews Correlation Coefficient (MCC): {mcc:.4f}")

In [ ]:
file_path = '/root/autodl-tmp/ROC/'
xgb_predicted_probabilities = np.load(file_path + 'XGB/y_val_pred.npy')  
svm_predicted_probabilities = np.load(file_path + 'SVM/y_val_pred.npy')  
rf_predicted_probabilities = np.load(file_path + 'RF/y_val_pred.npy')  
lstm_predicted_probabilities = np.load(file_path + 'LSTM/y_val_pred.npy')
transformer_predicted_probabilities = np.load(file_path + 'Transformer/y_val_pred.npy')  
true_labels = np.load(file_path + 'XGB/y_val_pred.npy') 

In [ ]:
np.array(rf_predicted_probabilities).shape

#### RF+Transformer

In [ ]:
predicted_probabilities = (np.array(rf_predicted_probabilities) + 
                           np.array(transformer_predicted_probabilities)
                           )/2
true_labels_ = np.argmax(true_labels, axis=-1)
predicted_labels = np.argmax(predicted_probabilities, axis=-1)
preds = torch.tensor(predicted_probabilities)
preds = F.softmax(preds, dim=-1)

test_metrics = calculate_multiclass_metrics(true_labels_, predicted_labels, preds, num_classes=3)

#### RF+Transformer+XGB

In [ ]:
predicted_probabilities = (np.array(rf_predicted_probabilities) + 
                           np.array(transformer_predicted_probabilities) + 
                           np.array(xgb_predicted_probabilities)
                           )/3
true_labels_ = np.argmax(true_labels, axis=-1)
predicted_labels = np.argmax(predicted_probabilities, axis=-1)
preds = torch.tensor(predicted_probabilities)
preds = F.softmax(preds, dim=-1)

test_metrics = calculate_multiclass_metrics(true_labels_, predicted_labels, preds, num_classes=3)

#### RF+Transformer+SVM

In [ ]:
predicted_probabilities = (np.array(rf_predicted_probabilities) + 
                           np.array(transformer_predicted_probabilities) + 
                           np.array(svm_predicted_probabilities)
                           )/3
true_labels_ = np.argmax(true_labels, axis=-1)
predicted_labels = np.argmax(predicted_probabilities, axis=-1)
preds = torch.tensor(predicted_probabilities)
preds = F.softmax(preds, dim=-1)

test_metrics = calculate_multiclass_metrics(true_labels_, predicted_labels, preds, num_classes=3)

#### RF+Transformer+XGB+SVM

In [ ]:
predicted_probabilities = (np.array(rf_predicted_probabilities) + 
                           np.array(transformer_predicted_probabilities) + 
                           np.array(xgb_predicted_probabilities) +
                           np.array(svm_predicted_probabilities)
                           )/4
true_labels_ = np.argmax(true_labels, axis=-1)
predicted_labels = np.argmax(predicted_probabilities, axis=-1)
preds = torch.tensor(predicted_probabilities)
preds = F.softmax(preds, dim=-1)

test_metrics = calculate_multiclass_metrics(true_labels_, predicted_labels, preds, num_classes=3)

#### RF+Transformer+XGB+SVM+LSTM

In [ ]:
predicted_probabilities = (np.array(rf_predicted_probabilities) + 
                           np.array(transformer_predicted_probabilities) + 
                           np.array(xgb_predicted_probabilities) +
                           np.array(svm_predicted_probabilities) +
                           np.array(lstm_predicted_probabilities)
                           )/5
true_labels_ = np.argmax(true_labels, axis=-1)
predicted_labels = np.argmax(predicted_probabilities, axis=-1)
preds = torch.tensor(predicted_probabilities)
preds = F.softmax(preds, dim=-1)

test_metrics = calculate_multiclass_metrics(true_labels_, predicted_labels, preds, num_classes=3)